In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [11]:
def capturar_anos(opcao):
    url = f'http://vitibrasil.cnpuv.embrapa.br/index.php?ano=2023&opcao={opcao}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        label_ano = soup.find('label', class_='lbl_pesq')
        if label_ano:
            texto = label_ano.get_text(strip=True) 
            print(f"🔎 Texto do label encontrado: {texto}")  

            anos = re.findall(r'\[(\d{4})-(\d{4})\]', texto)
            if anos:
                ano_inicio, ano_fim = map(int, anos[0])
                print(f"📅 Intervalo de anos detectado: {ano_inicio} - {ano_fim}")
                return ano_inicio, ano_fim
            else:
                print("⚠️ Regex não encontrou o intervalo de anos.")
                return None, None
        else:
            print("❌ Elemento <label class='lbl_pesq'> não encontrado.")
            return None, None

    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return None, None

In [16]:
teste, teste1 = map(int, capturar_anos('opt_02'))
print(teste, teste1)

🔎 Texto do label encontrado: Ano: [1970-2023]
📅 Intervalo de anos detectado: 1970 - 2023
1970 2023


In [4]:
def capturar_subopcoes(opcao):
    """
    Descobre dinamicamente os subopções (ex: subopt_01) e seus nomes amigáveis para uma dada opção.
    """
    url = f'http://vitibrasil.cnpuv.embrapa.br/index.php?ano=2023&opcao={opcao}'
    subopcoes = {}

    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')


        botoes = soup.find_all('button', {'name': 'subopcao'})
        for botao in botoes:
            valor = botao.get('value')
            nome = botao.get_text(strip=True)
            if valor and nome:
                subopcoes[valor] = nome

    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar subopções de {opcao}: {e}")

    return subopcoes


In [10]:
teste = capturar_subopcoes('opt_05')
print(teste)

{'subopt_01': 'Vinhos de mesa', 'subopt_02': 'Espumantes', 'subopt_03': 'Uvas frescas', 'subopt_04': 'Uvas passas', 'subopt_05': 'Suco de uva'}


In [2]:
def capturar_dados_opt_02_04(ano_inicio, ano_fim):
    """
    Coleta e interpreta os dados da tabela HTML da opção opt_02 e opt_04 do site da Embrapa.,
    estruturando produtos e subprodutos com seus respectivos valores.
    """
    dados = []

    for ano in range(ano_inicio, ano_fim + 1):
        print(f"\n📥 Coletando dados para o ano {ano}, opção opt_02...")
        url = f'http://vitibrasil.cnpuv.embrapa.br/index.php?ano={ano}&opcao=opt_02'

        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            tabela = soup.find('table', class_='tb_base tb_dados')
            if not tabela:
                print(f"⚠️ Nenhuma tabela válida encontrada para o ano {ano}.")
                continue

            produto_atual = None

            for row in tabela.find_all('tr'):
                cols = row.find_all('td')

                if not cols or len(cols) < 2:
                    continue

                primeira_coluna = cols[0].get_text(strip=True)
                segunda_coluna = cols[1].get_text(strip=True)


                if not primeira_coluna or 'total' in primeira_coluna.lower():
                    continue

                if segunda_coluna == '-' or segunda_coluna.strip() == '':
                    valor = None
                else:
                    try:
                        valor = float(segunda_coluna.replace('.', '').replace(',', '.'))
                    except ValueError:
                        valor = None


                if 'tb_item' in cols[0].get('class', []):
                    produto_atual = primeira_coluna
                    dados.append({
                        'ano': ano,
                        'produto': produto_atual,
                        'subproduto': 'Valor Total Produto',
                        'quantidade_l': valor
                    })

                elif 'tb_subitem' in cols[0].get('class', []) and produto_atual:
                    subproduto = primeira_coluna.lower()
                    dados.append({
                        'ano': ano,
                        'produto': produto_atual,
                        'subproduto': subproduto,
                        'quantidade_l': valor
                    })

        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar {url}: {e}")
            continue

    return pd.DataFrame(dados)


In [ ]:
def capturar_dados_opt_03(ano_inicio, ano_fim):
    """
    Coleta os dados de todas as subopções da opção opt_03,
    estruturando cultivar, subproduto e tipo de comercialização com respectivos valores.
    """
    dados = []
    subopcoes = capturar_subopcoes('opt_03')

    for ano in range(ano_inicio, ano_fim + 1):
        for sub, tipo_nome in subopcoes.items():
            print(f"\n📥 Coletando dados para ano {ano}, subopção {sub} ({tipo_nome})...")
            url = f'http://vitibrasil.cnpuv.embrapa.br/index.php?ano={ano}&opcao=opt_03&subopcao={sub}'

            try:
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'html.parser')

                tabela = soup.find('table', class_='tb_base tb_dados')
                if not tabela:
                    print(f"⚠️ Nenhuma tabela válida encontrada para o ano {ano}, {sub}.")
                    continue

                cultivar_atual = None

                for row in tabela.find_all('tr'):
                    cols = row.find_all('td')

                    if not cols or len(cols) < 2:
                        continue

                    primeira_coluna = cols[0].get_text(strip=True)
                    segunda_coluna = cols[1].get_text(strip=True)

                    if not primeira_coluna or 'total' in primeira_coluna.lower():
                        continue

                    if segunda_coluna == '-' or segunda_coluna.strip() == '':
                        valor = None
                    else:
                        try:
                            valor = float(segunda_coluna.replace('.', '').replace(',', '.'))
                        except ValueError:
                            valor = None

                    if 'tb_item' in cols[0].get('class', []):
                        cultivar_atual = primeira_coluna
                        dados.append({
                            'ano': ano,
                            'tipo': tipo_nome,
                            'cultivar': cultivar_atual,
                            'subproduto': 'valor total cultivar',
                            'quantidade_kg': valor
                        })

                    elif 'tb_subitem' in cols[0].get('class', []) and cultivar_atual:
                        subproduto = primeira_coluna.lower()
                        dados.append({
                            'ano': ano,
                            'tipo': tipo_nome,
                            'cultivar': cultivar_atual,
                            'subproduto': subproduto,
                            'quantidade_kg': valor
                        })

            except requests.exceptions.RequestException as e:
                print(f"Erro ao acessar {url}: {e}")
                continue

    return pd.DataFrame(dados)


In [ ]:
def capturar_dados_opt_05_06(opcao, ano_inicio, ano_fim):
    """
    Raspagem automatizada para opt_05 e opt_06, com subopções e estrutura de exportação (país, quantidade, valor).
    Inclui logs para ajudar a depurar problemas com dados NaN.
    """
    dados = []
    subopcoes = capturar_subopcoes(opcao)

    for ano in range(ano_inicio, ano_fim + 1):
        for sub, tipo_nome in subopcoes.items():
            print(f"\n📥 Coletando dados para {opcao} - {tipo_nome} ({sub}) - Ano {ano}")
            url = f'http://vitibrasil.cnpuv.embrapa.br/index.php?ano={ano}&opcao={opcao}&subopcao={sub}'

            try:
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'html.parser')

                tabela = soup.find('table', class_='tb_base tb_dados')
                if not tabela:
                    print(f"⚠️ Tabela não encontrada para {ano}, {opcao}, {sub}")
                    continue

                for row in tabela.find_all('tr'):
                    cols = row.find_all('td')

                    if len(cols) != 3:
                        print(f"❌ Ignorando linha com {len(cols)} colunas: {[col.get_text(strip=True) for col in cols]}")
                        continue

                    pais = cols[0].get_text(strip=True)
                    quantidade_raw = cols[1].get_text(strip=True)
                    valor_raw = cols[2].get_text(strip=True)

                    print(f"➡️ País: {pais}, Quantidade (bruto): '{quantidade_raw}', Valor (bruto): '{valor_raw}'")

                    try:
                        quantidade = float(quantidade_raw.replace('.', '').replace(',', '.')) \
                            if quantidade_raw and quantidade_raw != '-' else None
                    except ValueError:
                        print(f"⚠️ Erro ao converter quantidade: {quantidade_raw}")
                        quantidade = None

                    try:
                        valor = float(valor_raw.replace('.', '').replace(',', '.')) \
                            if valor_raw and valor_raw != '-' else None
                    except ValueError:
                        print(f"⚠️ Erro ao converter valor: {valor_raw}")
                        valor = None

                    dados.append({
                        'ano': ano,
                        'tipo': tipo_nome,
                        'pais': pais,
                        'quantidade_kg': quantidade,
                        'valor_usd': valor
                    })

            except requests.exceptions.RequestException as e:
                print(f"Erro ao acessar {url}: {e}")
                continue

    return pd.DataFrame(dados)


In [7]:
def capturar_dados_opt_02(ano_inicio, ano_fim):
    return capturar_dados_opt_02_04(ano_inicio, ano_fim)

def capturar_dados_opt_04(ano_inicio, ano_fim):
    return capturar_dados_opt_02_04(ano_inicio, ano_fim)

def capturar_dados_opt_05(ano_inicio, ano_fim):
    return capturar_dados_opt_05_06('opt_05', ano_inicio, ano_fim)

def capturar_dados_opt_06(ano_inicio, ano_fim):
    return capturar_dados_opt_05_06('opt_06', ano_inicio, ano_fim)


In [8]:
def capturar_dados(opcao, ano_inicio, ano_fim):
    if opcao == 'opt_02':
        return capturar_dados_opt_02(ano_inicio, ano_fim)
    elif opcao == 'opt_03': 
        return capturar_dados_opt_03(ano_inicio, ano_fim)
    elif opcao == 'opt_04':
        return capturar_dados_opt_04(ano_inicio, ano_fim)
    elif opcao == 'opt_05':
        return capturar_dados_opt_05(ano_inicio, ano_fim)
    elif opcao == 'opt_06':
        return capturar_dados_opt_06(ano_inicio, ano_fim)
    else:
        print(f"⚠️ Nenhum parser implementado para a opção {opcao}")
        return pd.DataFrame()


In [9]:
def main():
    opcoes = [
        'opt_02',
        'opt_03',
        'opt_04',
        'opt_05',
        'opt_06'
        ]
    dados_opcoes = {}

    for idx, opcao in enumerate(opcoes, start=1):
        print(f'\n🚀 Processando opção {idx}/{len(opcoes)}: {opcao}')
        ano_inicio, ano_fim = capturar_anos(opcao)

        if ano_inicio and ano_fim:
            df = capturar_dados(opcao, ano_inicio, ano_fim)
            if not df.empty:
                dados_opcoes[opcao] = df
                print(f"✅ Dados coletados para {opcao}. {len(df)} registros.")
            else:
                print(f"⚠️ Nenhum dado encontrado para {opcao}.")
        else:
            print(f"❌ Intervalo de anos não encontrado para {opcao}.")

    for opcao, df in dados_opcoes.items():
        print(f"\n🔎 Amostra dos dados da opção {opcao}:")
        print(df.head())

In [19]:
df = capturar_dados('opt_06', 2020, 2023)
df = pd.DataFrame(df)


📥 Coletando dados para opt_06 - Vinhos de mesa (subopt_01) - Ano 2020
❌ Ignorando linha com 0 colunas: []
➡️ País: Afeganistão, Quantidade (bruto): '-', Valor (bruto): '-'
➡️ País: África do Sul, Quantidade (bruto): '4', Valor (bruto): '21'
➡️ País: Alemanha, República Democrática, Quantidade (bruto): '6.261', Valor (bruto): '32.605'
➡️ País: Angola, Quantidade (bruto): '-', Valor (bruto): '-'
➡️ País: Anguilla, Quantidade (bruto): '-', Valor (bruto): '-'
➡️ País: Antígua e Barbuda, Quantidade (bruto): '624', Valor (bruto): '1.864'
➡️ País: Antilhas Holandesas, Quantidade (bruto): '-', Valor (bruto): '-'
➡️ País: Arábia Saudita, Quantidade (bruto): '-', Valor (bruto): '-'
➡️ País: Argélia, Quantidade (bruto): '-', Valor (bruto): '-'
➡️ País: Argentina, Quantidade (bruto): '1.015', Valor (bruto): '4.176'
➡️ País: Aruba, Quantidade (bruto): '-', Valor (bruto): '-'
➡️ País: Austrália, Quantidade (bruto): '1.013', Valor (bruto): '3.413'
➡️ País: Áustria, Quantidade (bruto): '-', Valor (br

In [20]:
df.head()

,ano,tipo,pais,quantidade_kg,valor_usd
0,2020,Vinhos de mesa,Afeganistão,NaN,NaN
1,2020,Vinhos de mesa,África do Sul,4.0,21.0
2,2020,Vinhos de mesa,"Alemanha, República Democrática",6261.0,32605.0
3,2020,Vinhos de mesa,Angola,NaN,NaN
4,2020,Vinhos de mesa,Anguilla,NaN,NaN


In [18]:
if __name__ == "__main__":
    main()


🚀 Processando opção 1/5: opt_02
🔎 Texto do label encontrado: Ano: [1970-2023]
📅 Intervalo de anos detectado: 1970 - 2023

📥 Coletando dados para o ano 1970, opção opt_02...

📥 Coletando dados para o ano 1971, opção opt_02...

📥 Coletando dados para o ano 1972, opção opt_02...

📥 Coletando dados para o ano 1973, opção opt_02...

📥 Coletando dados para o ano 1974, opção opt_02...

📥 Coletando dados para o ano 1975, opção opt_02...

📥 Coletando dados para o ano 1976, opção opt_02...

📥 Coletando dados para o ano 1977, opção opt_02...

📥 Coletando dados para o ano 1978, opção opt_02...

📥 Coletando dados para o ano 1979, opção opt_02...

📥 Coletando dados para o ano 1980, opção opt_02...

📥 Coletando dados para o ano 1981, opção opt_02...

📥 Coletando dados para o ano 1982, opção opt_02...

📥 Coletando dados para o ano 1983, opção opt_02...

📥 Coletando dados para o ano 1984, opção opt_02...

📥 Coletando dados para o ano 1985, opção opt_02...

📥 Coletando dados para o ano 1986, opção opt_

KeyboardInterrupt: 

In [14]:
df = capturar_dados_opt_05(2020, 2023)
print(df)

Erro ao acessar subopções de opt_05: HTTPConnectionPool(host='vitibrasil.cnpuv.embrapa.br', port=80): Max retries exceeded with url: /index.php?ano=2023&opcao=opt_05 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000016CE2F0B9B0>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
Empty DataFrame
Columns: []
Index: []
